In [1]:
from datasets import load_dataset
import pandas as pd

/scratch/rahul/miniconda3/envs/nlp-train/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
data_base_path = "/scratch/rahul/nlp/augsnli_1.0/"
dataset = load_dataset("csv", data_files={"train": data_base_path+"train.tsv", "validation": data_base_path+"dev.tsv", "test": data_base_path+"test.tsv"}, sep='\t', usecols=["Unnamed: 0", "index", "captionID", "sentence1", "sentence2", "gold_label"])
dataset = dataset.rename_column("gold_label", "label")
dataset = dataset.rename_column("sentence1", "premise")
dataset = dataset.rename_column("sentence2", "hypothesis")
dataset = dataset.rename_column("Unnamed: 0", "ct_idx")
dataset = dataset.rename_column("index", "og_idx")
dataset = dataset.rename_column("captionID", "trans")
dataset = dataset.filter(lambda ex: ex["hypothesis"] is not None)
def rename_trans(ex):
    if ex["trans"].startswith("*"):
        ex["trans"] = "original"
    return ex
dataset = dataset.map(rename_trans)

Map: 100%|██████████| 14362/14362 [00:06<00:00, 2299.33 examples/s] 


In [37]:
df = dataset["test"].to_pandas()
df

,ct_idx,og_idx,trans,premise,hypothesis,label
0,0,0,original,"A person walks outside carrying a bag, while a...","A woman carrying a girl, with a bag behind her",contradiction
1,1,1,original,A dark-haired man in a black uniform is speaki...,A man in a police uniform is talking to anothe...,neutral
2,2,2,original,A middle-aged woman in a light brown trench co...,The middle aged woman is at home knitting.,contradiction
3,3,3,original,One man in baseball cap looks out while two me...,Women look different directions.,contradiction
4,4,4,original,A child in an orange outfit stands holding flo...,The boy has a gift for his mother.,neutral
...,...,...,...,...,...,...
14357,14564,9996,original,A city scene is being painted by a man on a ca...,Some dolphins are being painted by a man.,neutral
14358,14565,9996,original,It is a man who is painting a city scene on a ...,It is a man who is painting some dolphins.,neutral
14359,14566,9997,original,People sitting in a dim restaurant eating.,People are in a restaurant celebrating an anni...,neutral
14360,14567,9998,original,A small girl is swinging.,a young lady plays outside,neutral


In [38]:
ogs = df["og_idx"].unique()
for i in ogs:
    filt = df[(df["og_idx"] == i)] 
    if len(filt) > 4:
        print(filt)
        break

    ct_idx  og_idx                                              trans  \
14      14      13                                           original   
15      15      13               entailment;past simple;future simple   
16      16      13                            entailment;may;original   
17      17      13                                 entailment;may;may   
18      18      13                                           original   
19      19      13  entailment;past simple+it cleft: ARG1;future s...   

                                              premise  \
14  Two workers stretch the canvas of the advertis...   
15  Two workers stretched the canvas of the advert...   
16  In order for free range chickens to be appende...   
17  In order for free range chickens to be appende...   
18  It is two workers which stretch the canvas of ...   
19  In order for free range chickens to be appende...   

                                 hypothesis       label  
14                   Two men w